In [1]:
import monai
import torch
from torch.nn.functional import softmax

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [4]:
transform=monai.transforms.Compose(
        [
            monai.transforms.LoadImage(
                image_only=True
            ),
            monai.transforms.EnsureChannelFirst(channel_dim = None),
            monai.transforms.Resize(
                spatial_size=(256,256,30),
                mode=("trilinear"),
            ),
            monai.transforms.ScaleIntensity(minv=0.0, maxv=1.0),
            monai.transforms.NormalizeIntensity()
        ]
    )
image = transform('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/input/images/axial-t2-prostate-mri/a55112de-6e99-4539-b587-5857729d6d51.mha')
image = image.unsqueeze(0).to(device)
model = monai.networks.nets.EfficientNetBN(model_name= 'efficientnet-b7', pretrained=False, progress=False, 
                                        spatial_dims=3, in_channels=1, num_classes=2)
model.load_state_dict(torch.load('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/model_best_weights.pth'))
model.to(device)
risk_score_likelihood = softmax(model(image),dim=1)[0][1].item()

OutOfMemoryError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 1.82 GiB total capacity; 989.93 MiB already allocated; 70.00 MiB free; 1012.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
transform=monai.transforms.Compose(
        [
            monai.transforms.LoadImage(
                image_only=True, reader = "NibabelReader"
            ),
            monai.transforms.EnsureChannelFirst(channel_dim = None),
            monai.transforms.Resize(
                spatial_size=(256,256,30),
                mode=("trilinear"),
            ),
            monai.transforms.ScaleIntensity(minv=0.0, maxv=1.0),
            monai.transforms.NormalizeIntensity()
        ]
    )
image = transform('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/a55112de-6e99-4539-b587-5857729d6d51.mha')
image = image.unsqueeze(0)

In [6]:
image.shape

torch.Size([1, 1, 256, 256, 30])

In [8]:
risk_score_likelihood[0][1].item()

1.650426747801248e-05

In [16]:
image.shape

torch.Size([1, 1, 256, 256, 30])